In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold,cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc,confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
import tkinter
import PIL.Image, PIL.ImageTk

In [2]:
df=pd.read_csv('data/final_data.csv')
del df['Unnamed: 0']
del df['boolDeadRelations']
del df['age']
del df['isNoble']

df.set_index('name',inplace=True)

# get dummy varibles for ordinal varibles
culture_dummy=pd.get_dummies(df['culture'],prefix='Culture') #Nan value will be ingored automatically
df[culture_dummy.columns]=culture_dummy
del df['culture']

house_dummy=pd.get_dummies(df['house'],prefix='House')
df[house_dummy.columns]=house_dummy
del df['house']

identity_dummy=pd.get_dummies(df['identity'])
df[identity_dummy.columns]=identity_dummy
del df['identity']  
del df['lord']  #delete one column to eliminate the linear relationships

#get the attribute set and label set
X=df.drop('isAlive',axis=1)
Y=df['isAlive']
df=df.drop('isAlive',axis=1)
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.15)

In [3]:
df.head()

male  book1  book2  book3  book4  book5  isMarried  \
name                                                                       
Viserys II Targaryen     1      0      0      0      0      0          0   
Walder Frey              1      1      1      1      1      1          1   
Addison Hill             1      0      0      0      1      0          0   
Aemma Arryn              0      0      0      0      0      0          1   
Sylva Santagar           0      0      0      0      1      0          1   

                      numDeadRelations  popularity  Culture_Andal  ...  \
name                                                               ...   
Viserys II Targaryen                11    0.605351              0  ...   
Walder Frey                          1    0.896321              0  ...   
Addison Hill                         0    0.267559              0  ...   
Aemma Arryn                          0    0.183946              0  ...   
Sylva Santagar                       0    0.043478              0  ...   

                      House_brotherhood without banners  House_wildling  king  \
name                                                                            
Viserys II Targaryen                                  0               0     1   
Walder Frey                                           0               0     0   
Addison Hill                                          0               0     0   
Aemma Arryn                                           0               0     0   
Sylva Santagar                                        0               0     0   

                      knight  magician  noble  official  prince  smallfolk  \
name                                                                         
Viserys II Targaryen       0         0      0         0       0          0   
Walder Frey                1         0      0         0       0          0   
Addison Hill               0         0      0         0       0          0   
Aemma Arryn                0         0      0         1       0          0   
Sylva Santagar             0         0      1         0       0          0   

                      warrior  
name                           
Viserys II Targaryen        0  
Walder Frey                 0  
Addison Hill                0  
Aemma Arryn                 0  
Sylva Santagar              0  

[5 rows x 382 columns]

In [4]:
df.loc[df['Culture_northmen']==1]

male  book1  book2  book3  book4  book5  isMarried  \
name                                                           
Turnip       0      0      1      0      1      0          0   
Varly        0      1      0      0      0      0          0   
Walton       1      0      1      1      1      1          0   
Heward       0      1      0      0      0      0          0   
Porther      1      1      1      0      0      0          0   
Poxy Tym     1      0      1      0      0      0          0   
Jacks        0      1      1      1      1      0          0   
Quent        1      1      1      1      1      0          0   
Tomard       1      1      1      1      0      1          0   

          numDeadRelations  popularity  Culture_Andal  ...  \
name                                                   ...   
Turnip                   0    0.016722              0  ...   
Varly                    0    0.016722              0  ...   
Walton                   0    0.046823              0  ...   
Heward                   0    0.020067              0  ...   
Porther                  0    0.010033              0  ...   
Poxy Tym                 0    0.010033              0  ...   
Jacks                    0    0.033445              0  ...   
Quent                    0    0.023411              0  ...   
Tomard                   0    0.063545              0  ...   

          House_brotherhood without banners  House_wildling  king  knight  \
name                                                                        
Turnip                                    0               0     0       0   
Varly                                     0               0     0       0   
Walton                                    0               0     0       0   
Heward                                    0               0     0       0   
Porther                                   0               0     0       0   
Poxy Tym                                  0               0     0       0   
Jacks                                     0               0     0       0   
Quent                                     0               0     0       0   
Tomard                                    0               0     0       0   

          magician  noble  official  prince  smallfolk  warrior  
name                                                             
Turnip           0      0         0       0          1        0  
Varly            0      0         0       0          1        0  
Walton           0      0         0       0          1        0  
Heward           0      0         0       0          1        0  
Porther          0      0         0       0          1        0  
Poxy Tym         0      0         0       0          1        0  
Jacks            0      0         0       0          1        0  
Quent            0      0         0       0          1        0  
Tomard           0      0         0       0          0        1  

[9 rows x 382 columns]

In [5]:
df.columns.shape

(382,)

In [6]:
single_df=pd.DataFrame(np.array([0]*382).reshape(1,382),columns=df.columns)

In [7]:
Logis_model=LogisticRegression(solver='liblinear').fit(X_train,Y_train)
y_pre=Logis_model.predict(X_test)
print(f'the accuracy of LogisticRegression is {np.mean(y_pre==Y_test)}')
Logis_model.predict_proba(single_df.values)

the accuracy of LogisticRegression is 0.7945205479452054


array([[0.28681996, 0.71318004]])

In [8]:
def calculate_DP():
    male=combo_gender.get()
    if male=='Male':
        single_df.loc[0]['male']=1
    elif male=='Female':
        single_df.loc[0]['male']=0
    marry=combo_marry.get()
    if marry=='Married':
        single_df.loc[0]['isMarried']=1
    else:
        single_df.loc[0]['isMarried']=0
    numRelation=spin.get()
    single_df.loc[0]['numDeadRelations']=numRelation
    pop=txt.get()
    single_df.loc[0]['popularity']=pop
    culture=combo_culture.get()
    single_df.loc[0][culture]=1
    house=combo_house.get()
    single_df.loc[0][house]=1
    identity=combo_identity.get()
    single_df.loc[0][identity]=1
    book1_bool=book1.get()
    if book1_bool:
        single_df.loc[0]['book1']=1
    book2_bool=book2.get()
    if book2_bool:
        single_df.loc[0]['book2']=1
    book3_bool=book3.get()
    if book3_bool:
        single_df.loc[0]['book3']=1
    book4_bool=book4.get()
    if book4_bool:
        single_df.loc[0]['book4']=1
    book5_bool=book5.get()
    if book5_bool:
        single_df.loc[0]['book5']=1
    txt_prob.insert(0,str(Logis_model.predict_proba(single_df.values).ravel()[1])[:5])

In [9]:
from tkinter import *
 
from tkinter.ttk import *
 
window = Tk()
 
window.title("Welcome to Your Adventure in GoT")
 
window.geometry('750x400')


img = PIL.ImageTk.PhotoImage(PIL.Image.open("images/photo.jpg"))
panel = Label(window, image = img)
panel.image=img
panel.grid(column=0,row=0,columnspan=10,rowspan=15)
#choose the gender
lbl_gender = Label(window, text="Gender")
 
lbl_gender.grid(column=0, row=0)

combo_gender = Combobox(window)
 
combo_gender['values']= ("Male","Female")
 
combo_gender.current(0) #set the selected item

combo_gender.grid(column=1, row=0,pady=5)

#choose the book where the character is from
   
book1 = BooleanVar() 
book1.set(False) #set check state 
chk1 = Checkbutton(window, text='Book1', var=book1)
chk1.grid(column=0,row=1,padx=20) #set the selected item

book2 = BooleanVar() 
book2.set(False) #set check state 
chk2 = Checkbutton(window, text='Book2', var=book2)
chk2.grid(column=1,row=1,padx=20) #set the selected item

book3 = BooleanVar() 
book3.set(False) #set check state 
chk3 = Checkbutton(window, text='Book3', var=book3)
chk3.grid(column=2, row=1,padx=20) #set the selected item

book4 = BooleanVar() 
book4.set(False) #set check state 
chk4 = Checkbutton(window, text='Book4', var=book4)
chk4.grid(column=3, row=1,padx=30) #set the selected item

book5 = BooleanVar() 
book5.set(False) #set check state 
chk5 = Checkbutton(window, text='Book5', var=book5)
chk5.grid(column=4, row=1,padx=30) #set the selected item

#choose wether the character is married
lbl_marry = Label(window, text="Marriage")
 
lbl_marry.grid(column=0, row=2)

combo_marry = Combobox(window)
 
combo_marry['values']= ("Married","Unmarried")
 
combo_marry.current(0) #set the selected item
 
combo_marry.grid(column=1, row=2,pady=5)

#choose number of dead relations ID
lbl_num = Label(window, text="NumRelations")
 
lbl_num.grid(column=0, row=3)

spin = Spinbox(window, from_=0, to=100, width=8)
 
spin.grid(column=1,row=3, pady=5)

#choose the popularity
lbl_pop = Label(window, text="Popularity (Between 0 and 1)")
 
lbl_pop.grid(column=0, row=4)

#lbl_spec = Label(window, text="(Between 0 and 1)")
 
#lbl_spec.grid(column=2, row=4)

txt = Entry(window,width=10)
 
txt.grid(column=1, row=4,pady=5)

#choose the culture
lbl_culture = Label(window, text="Culture")
 
lbl_culture.grid(column=0, row=5)

combo_culture = Combobox(window)
 
combo_culture['values']= (list(culture_dummy.columns))

combo_culture.current(0) #set the selected item
 
combo_culture.grid(column=1, row=5,pady=5)

#choose the house
lbl_house = Label(window, text="House")
 
lbl_house.grid(column=0, row=6)

combo_house = Combobox(window)
 
combo_house['values']= (list(house_dummy.columns))
 
combo_house.current(0) #set the selected item
 
combo_house.grid(column=1, row=6,pady=5)

#choose the identity
lbl_identity = Label(window, text="Identity")
 
lbl_identity.grid(column=0, row=7)

combo_identity = Combobox(window)
 
combo_identity['values']= (list(identity_dummy.columns))
 
combo_identity.current(0) #set the selected item
 
combo_identity.grid(column=1, row=7,pady=5)

#add button
button=Button(window,text='submit',command=calculate_DP)
button.grid(column=0,row=8,pady=20)

#add the identity
lbl_prob = Label(window, text="Probability")
 
lbl_prob.grid(column=0, row=9)

txt_prob = Entry(window)
 
txt_prob.grid(column=1, row=9,pady=5)

window.mainloop()